<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#소-가격-크롤링" data-toc-modified-id="소-가격-크롤링-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>소 가격 크롤링</a></span></li><li><span><a href="#돼지-가격-크롤링" data-toc-modified-id="돼지-가격-크롤링-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>돼지 가격 크롤링</a></span></li><li><span><a href="#local-mysql에-적재하는-코드-짜놓기" data-toc-modified-id="local-mysql에-적재하는-코드-짜놓기-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>local mysql에 적재하는 코드 짜놓기</a></span><ul class="toc-item"><li><span><a href="#소-DB에-적재하는-코드" data-toc-modified-id="소-DB에-적재하는-코드-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>소 DB에 적재하는 코드</a></span></li><li><span><a href="#돼지-DB에-적재" data-toc-modified-id="돼지-DB에-적재-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>돼지 DB에 적재</a></span></li></ul></li></ul></div>

- 출처 : 축산물품질평가원 공공 데이터 포탈 API이용

- <소>
    - 소 권역별 경락가격 현황
- <돼지>
    - 돼지 박피,탕박 권역별 경락가격 현황


In [57]:
# 한 번에 가져오는 건 일일 트래픽 1000을 초과하기 때문에 2년치 씩만 가져와서 DB에 적재하자..
# 오늘 밤 12시 지나서 하기

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [16]:
beef_url = 'http://data.ekape.or.kr/openapi-data/service/user/grade/auct/cattleApperence?baseYmd=2017-09-10&ServiceKey=0OhBU7ZCGIobDVKDeBJDpmDRqK3IRNF6jlf%2FJB2diFAf%2FfR2czYO9A4UTGcsOwppV6W2HVUeho%2FFPwXoL6DwqA%3D%3D'
pork_url = 'http://data.ekape.or.kr/openapi-data/service/user/grade/auct/pigApperence?baseYmd=20170908&ServiceKey=0OhBU7ZCGIobDVKDeBJDpmDRqK3IRNF6jlf%2FJB2diFAf%2FfR2czYO9A4UTGcsOwppV6W2HVUeho%2FFPwXoL6DwqA%3D%3D'

## 소 가격 크롤링

In [ ]:
beef_dict = {}

dates = []
beefs = []
beef_ks = []
regions = []
prices = []

datetimes = pd.date_range(start='2014-01-08', end='', freq='D').strftime('%Y-%m-%d').tolist()

for datetime in datetimes:
    beef_url = f'http://data.ekape.or.kr/openapi-data/service/user/grade/auct/cattleApperence?baseYmd={datetime}&ServiceKey=0OhBU7ZCGIobDVKDeBJDpmDRqK3IRNF6jlf%2FJB2diFAf%2FfR2czYO9A4UTGcsOwppV6W2HVUeho%2FFPwXoL6DwqA%3D%3D'
    res = requests.get(beef_url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.select('item')
    print(datetime)
    for content in contents:
        date = content.select_one('baseymd').get_text()
        beef = content.select_one('judgekindnm').get_text()
        beef_k = content.select_one('judgebreednm').get_text()
        region = content.select_one('localnm').get_text()
        price = content.select_one('auctamt').get_text()
        
        dates.append(date)
        beefs.append(beef)
        beef_ks.append(beef_k)
        regions.append(region)
        prices.append(price)

beef_dict['날짜'] = dates
beef_dict['동물'] = beefs
beef_dict['종류'] = beef_ks
beef_dict['지역'] = regions
beef_dict['가격'] = prices

In [39]:
df = pd.DataFrame(data_dict)
df.shape

(1594, 5)

---

## 돼지 가격 크롤링

- 2013년 2월 8일부터 크롤링해서 적재하면 댐!

In [ ]:
pork_dict = {}

dates = []
porks = []
pork_ks = []
regions = []
prices = []

datetimes = pd.date_range(start='2013-02-08', end='', freq='D').strftime('%Y%m%d').tolist()
for datetime in datetimes:
    pork_url = f'http://data.ekape.or.kr/openapi-data/service/user/grade/auct/pigApperence?baseYmd={datetime}&ServiceKey=0OhBU7ZCGIobDVKDeBJDpmDRqK3IRNF6jlf%2FJB2diFAf%2FfR2czYO9A4UTGcsOwppV6W2HVUeho%2FFPwXoL6DwqA%3D%3D'
    res = requests.get(pork_url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.select('item')
    print(datetime)
    for content in contents:
        date = content.select_one('baseymd').get_text()
        pork = content.select_one('judgekindnm').get_text()
        pork_k = content.select_one('skinnm').get_text()
        region = content.select_one('localnm').get_text()
        price = content.select_one('auctamt').get_text()
        
        dates.append(date)
        porks.append(pork)
        pork_ks.append(pork_k)
        regions.append(region)
        prices.append(price)
    
pork_dict['날짜'] = dates
pork_dict['동물'] = porks
pork_dict['종류'] = pork_ks
pork_dict['지역'] = regions
pork_dict['가격'] = prices

In [51]:
df = pd.DataFrame(pork_dict)
df.shape

(172, 5)

## local mysql에 적재하는 코드 짜놓기

- 2015년 01월 07일 까지 DB 적재 완료 상태!

### 소 DB에 적재하는 코드

In [40]:
import pymysql
db = pymysql.connect(host='localhost',
                    port=3306,
                    user='younghun',
                    password='watson1259',
                    db='beef_pork_db',
                    charset='utf8')
cursor = db.cursor()

rows = map(list, zip(*beef_dict.values()))

# 한 row씩 DB에 insert하기
for row in rows:
    sql = """INSERT INTO beef_prices VALUES('"""+row[0]+"""','"""+row[1]+"""','"""+row[2]+"""',
    '"""+str(row[3])+"""','"""+str(row[4])+"""')"""
    cursor.execute(sql)
    db.commit()
    
db.close()

### 돼지 DB에 적재

In [52]:
import pymysql
db = pymysql.connect(host='localhost',
                    port=3306,
                    user='younghun',
                    password='watson1259',
                    db='beef_pork_db',
                    charset='utf8')
cursor = db.cursor()

rows = map(list, zip(*pork_dict.values()))

# 한 row씩 DB에 insert하기
for row in rows:
    sql = """INSERT INTO pork_prices VALUES('"""+row[0]+"""','"""+row[1]+"""','"""+row[2]+"""',
    '"""+str(row[3])+"""','"""+str(row[4])+"""')"""
    cursor.execute(sql)
    db.commit()
    
db.close()

---